<a href="https://colab.research.google.com/github/jeramirez169/DataScience_SGS_Classification/blob/main/models/02_classic_models.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [33]:
# Clonar tu repositorio desde GitHub
!git clone https://github.com/jeramirez169/DataScience_SGS_Classification.git
%cd DataScience_SGS_Classification

# Instalar dependencias necesarias
!pip install -q pandas numpy scikit-learn unidecode spacy imbalanced-learn transformers
!python -m spacy download es_core_news_lg


Cloning into 'DataScience_SGS_Classification'...
remote: Enumerating objects: 74, done.
remote: Counting objects: 100% (74/74), done.
remote: Compressing objects: 100% (70/70), done.
remote: Total 74 (delta 30), reused 0 (delta 0), pack-reused 0 (from 0)
Receiving objects: 100% (74/74), 23.94 MiB | 18.42 MiB/s, done.
Resolving deltas: 100% (30/30), done.
/content/DataScience_SGS_Classification/DataScience_SGS_Classification
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 568.0/568.0 MB 757.3 kB/s eta 0:00:00
✔ Download and installation successful
You can now load the package via spacy.load('es_core_news_lg')
⚠ Restart to reload dependencies
If you are in a Jupyter or Colab notebook, you may need to restart Python in
order to load all the package's dependencies. You can do this by selecting the
'Restart kernel' or 'Restart runtime' option.


In [34]:
# Ir a la carpeta data
%cd data

# Descomprimir el archivo ZIP
!unzip "Dataset_SGS_clean.zip" -d .

# Regresar al directorio raíz del proyecto
%cd ..


/content/DataScience_SGS_Classification/DataScience_SGS_Classification/data
Archive:  Dataset SGS.zip
  inflating: ./Dataset SGS.csv       
/content/DataScience_SGS_Classification/DataScience_SGS_Classification


In [35]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns

from sklearn.model_selection import train_test_split
from sklearn.feature_extraction.text import TfidfVectorizer
from sklearn.svm import LinearSVC
from sklearn.preprocessing import LabelEncoder
from sklearn.metrics import classification_report, confusion_matrix
from xgboost import XGBClassifier
import joblib

sns.set(style="whitegrid")

# Carga del dataset limpio
ruta = "data/Dataset_SGS_clean.csv"   # Ajusta si tu ruta es distinta
df = pd.read_csv(ruta, encoding="utf-8")

print("Columnas del dataset:", df.columns.tolist())
print("Tamaño del dataset:", df.shape)

#Definición de variables X (texto) e y (etiqueta)
X = df["texto_truncado_lematizado"]
y = df["Oficina"]

# 3. Partición Train/Test (para ambos modelos)
X_train, X_test, y_train, y_test = train_test_split(
    X,
    y,
    test_size=0.20,
    stratify=y,
    random_state=42
)

print("Tamaño entrenamiento:", X_train.shape[0])
print("Tamaño prueba:", X_test.shape[0])

#
# Vectorización TF-IDF
tfidf = TfidfVectorizer(
    ngram_range=(1, 2),
    min_df=3,
    max_df=0.95,
    sublinear_tf=True
)

X_train_tfidf = tfidf.fit_transform(X_train)
X_test_tfidf = tfidf.transform(X_test)

print("Forma de la matriz TF-IDF (train):", X_train_tfidf.shape)

# MODELO SVM
svm_model = LinearSVC(
    C=1.0,
    loss="squared_hinge",
    class_weight="balanced",
    max_iter=5000,
    random_state=42
)

svm_model.fit(X_train_tfidf, y_train)

y_pred_svm = svm_model.predict(X_test_tfidf)

print("\n" + "="*60)
print("RESULTADOS SVM")
print("="*60)
print(classification_report(y_test, y_pred_svm))

# Matriz de confusión SVM
labels = sorted(y.unique())

cm_svm = confusion_matrix(y_test, y_pred_svm, labels=labels)

plt.figure(figsize=(10,7))
sns.heatmap(cm_svm, annot=True, fmt="d", cmap="Blues",
            xticklabels=labels, yticklabels=labels)
plt.title("Matriz de Confusión – SVM")
plt.xlabel("Predicción")
plt.ylabel("Real")
plt.xticks(rotation=45)
plt.yticks(rotation=0)
plt.tight_layout()
plt.show()

#MODELO XGBOOST

# Codificación de etiquetas
le = LabelEncoder()
y_train_enc = le.fit_transform(y_train)
y_test_enc = le.transform(y_test)

print("\nMapeo de etiquetas (clase → código):")
for cls, code in zip(le.classes_, le.transform(le.classes_)):
    print(f"{cls:25s} -> {code}")

# Definición del modelo XGBoost
xgb_model = XGBClassifier(
    objective="multi:softprob",
    num_class=len(le.classes_),
    max_depth=6,
    learning_rate=0.1,
    n_estimators=300,
    subsample=0.9,
    colsample_bytree=0.9,
    reg_alpha=1.0,
    reg_lambda=1.0,
    tree_method="gpu_hist",
    predictor="gpu_predictor",
    eval_metric="mlogloss",
    random_state=42
)

print("\nEntrenando XGBoost (puede tardar varios minutos)...")
xgb_model.fit(X_train_tfidf, y_train_enc)

y_pred_xgb_enc = xgb_model.predict(X_test_tfidf)
y_pred_xgb = le.inverse_transform(y_pred_xgb_enc)

print("\n" + "="*60)
print("RESULTADOS XGBOOST")
print("="*60)
print(classification_report(y_test, y_pred_xgb))

# Matriz de confusión XGBoost
cm_xgb = confusion_matrix(y_test, y_pred_xgb, labels=labels)

plt.figure(figsize=(10,7))
sns.heatmap(cm_xgb, annot=True, fmt="d", cmap="Greens",
            xticklabels=labels, yticklabels=labels)
plt.title("Matriz de Confusión – XGBoost")
plt.xlabel("Predicción")
plt.ylabel("Real")
plt.xticks(rotation=45)
plt.yticks(rotation=0)
plt.tight_layout()
plt.show()

# Matriz de confusión normalizada XGBoost
cm_xgb_norm = cm_xgb.astype("float") / cm_xgb.sum(axis=1)[:, np.newaxis]

plt.figure(figsize=(10,7))
sns.heatmap(cm_xgb_norm, annot=True, fmt=".2f", cmap="Greens",
            xticklabels=labels, yticklabels=labels)
plt.title("Matriz de Confusión Normalizada – XGBoost")
plt.xlabel("Predicción")
plt.ylabel("Real")
plt.xticks(rotation=45)
plt.yticks(rotation=0)
plt.tight_layout()
plt.show()

#Guardado de modelos y vectorizador
joblib.dump(tfidf, "models/tfidf_vectorizer.pkl")
joblib.dump(svm_model, "models/svm_model.pkl")
joblib.dump(xgb_model, "models/xgb_model_gpu.pkl")

print("\nModelos y vectorizador guardados en carpeta 'models/'.")
